In [49]:
import math
import os

import altair as alt

import numpy as np

import pandas as pd

import scipy.stats

import yaml

import pickle


In [2]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

if os.getcwd() == '/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/':
    pass
    print("Already in correct directory")
else:
    os.chdir("/fh/fast/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/")
    print("Setup in correct directory")
with open("config.yaml") as f:
    config = yaml.safe_load(f)

Setup in correct directory


In [116]:
validation_ic50s = pd.read_csv('/home/blarsen/bloom_j/computational_notebooks/blarsen/2023/scripts/new_pipe_analyses/IC50_update.csv',na_filter=None)


In [123]:
escape = pd.read_csv('/home/blarsen/bloom_j/computational_notebooks/blarsen/2023/Nipah_Malaysia_RBP_DMS/results/antibody_escape/averages/nAH1.3_mut_effect.csv')
display(escape)

,epitope,site,wildtype,mutant,mutation,escape_mean,escape_median,n_models,times_seen,frac_models,LibA-230725-nAH1.3,LibB-230720-nAH1.3
0,1,71,Q,A,Q71A,0.13040,0.13040,2,2.5,1.0,0.288000,-0.027210
1,1,71,Q,C,Q71C,-0.37010,-0.37010,2,3.5,1.0,-0.303100,-0.437200
2,1,71,Q,D,Q71D,-0.10720,-0.10720,2,4.5,1.0,-0.156200,-0.058260
3,1,71,Q,E,Q71E,-0.05127,-0.05127,2,4.0,1.0,-0.117100,0.014550
4,1,71,Q,F,Q71F,-0.07334,-0.07334,2,3.0,1.0,0.013060,-0.159700
...,...,...,...,...,...,...,...,...,...,...,...,...
10101,1,602,T,S,T602S,0.07911,0.07911,2,6.0,1.0,0.002504,0.155700
10102,1,602,T,V,T602V,0.07129,0.07129,2,6.5,1.0,0.126600,0.016000
10103,1,602,T,W,T602W,0.03724,0.03724,2,7.0,1.0,0.073830,0.000646
10104,1,602,T,Y,T602Y,0.17210,0.17210,2,7.0,1.0,0.111400,0.232700


In [126]:
merged = validation_ic50s.merge(escape,on=['mutation'])
display(merged)
merged.to_csv('../scripts/new_pipe_analyses/IC50_validations_merged.csv', index=False)

,antibody,mutation,measured IC50,lower_bound,epitope,site,wildtype,mutant,escape_mean,escape_median,n_models,times_seen,frac_models,LibA-230725-nAH1.3,LibB-230720-nAH1.3
0,nAH1.3,Y445M,0.006015,False,1,445,Y,M,0.000292,0.000292,2,7.0,1.0,0.0167,-0.01612
1,nAH1.3,L184H,0.010802,False,1,184,L,H,0.438400,0.438400,2,5.5,1.0,0.2842,0.59270
2,nAH1.3,Q450S,0.006540,False,1,450,Q,S,0.255300,0.255300,2,4.0,1.0,0.2942,0.21630
3,nAH1.3,W519T,0.007923,False,1,519,W,T,0.268500,0.268500,2,9.5,1.0,0.2460,0.29100
4,nAH1.3,I520T,8.700000,False,1,520,I,T,2.439000,2.439000,2,8.0,1.0,0.6869,4.19100
5,nAH1.3,D468V,0.487700,False,1,468,D,V,2.806000,2.806000,2,8.5,1.0,0.9184,4.69400
6,nAH1.3,I517K,10.000000,True,1,517,I,K,2.700000,2.700000,2,8.5,1.0,0.9274,4.47200
7,nAH1.3,P185D,10.000000,True,1,185,P,D,2.831000,2.831000,2,10.0,1.0,0.7472,4.91500


In [132]:
merged = pd.read_csv('/home/blarsen/bloom_j/computational_notebooks/blarsen/2023/scripts/new_pipe_analyses/IC50_validations_merged.csv',na_filter=None)

corr_chart = (
    alt.Chart(merged)
    .encode(
        x=alt.X(
            "measured IC50",
            title="measured IC50 (ug/ml)",
            scale=alt.Scale(type="log"),
            axis=alt.Axis(grid=False,tickCount=4)
        ),
        y=alt.Y(
            "escape_mean",
            title="Mean Escape",
            #scale=alt.Scale(type="log"),
            axis=alt.Axis(grid=False,tickCount=4),
        ),
        #facet=alt.Facet("antibody", columns=4, title=None),
        color=alt.Color('mutation',title='Mutant'),
        shape=alt.Shape('lower_bound',title='Lower Bound'),
    )
    .mark_point(filled=True, size=200, opacity=0.6)
    .configure_axis(grid=False)
    .resolve_scale(y="independent", x="independent")
    .properties(width=400, height=400)
)

corr_chart = corr_chart.configure_title(fontSize=24, font="Helvetica Light").configure_axis(
    labelFontSize=18,
    titleFontSize=20,
    labelFont="Helvetica Light",
    titleFont="Helvetica Light"
).configure_legend(
    titleFontSize=18,
    labelFontSize=16,
    titleFont="Helvetica Light",
    labelFont="Helvetica Light"
)

#corr


corr_chart

alt.Chart(...)

In [91]:
fold_changes = (
    validation_vs_prediction
    .rename(columns={"median_IC50": "predicted IC50"})
    .query("aa_substitutions != ''")
    [["antibody", "aa_substitutions", "measured IC50", "predicted IC50", "lower_bound"]]
    .merge(
        validation_vs_prediction
        .rename(columns={"median_IC50": "predicted IC50"})
        .query("aa_substitutions == ''")
        [["antibody", "measured IC50", "predicted IC50"]],
        on="antibody",
        how="left",
        validate="many_to_one",
        suffixes=[" mutant", " unmutated"],
    )
    .assign(
        measured_fold_change=lambda x: x["measured IC50 mutant"] / x["measured IC50 unmutated"],
        predicted_fold_change=lambda x: x["predicted IC50 mutant"] / x["predicted IC50 unmutated"],
    )
)

fold_changes

,antibody,aa_substitutions,measured IC50 mutant,predicted IC50 mutant,lower_bound,measured IC50 unmutated,predicted IC50 unmutated,measured_fold_change,predicted_fold_change
0,nAH1.3,D468V,0.487700,20.165210,False,0.006834,0.402963,71.360504,50.042395
1,nAH1.3,I517K,10.000000,16.269555,True,0.006834,0.402963,1463.204861,40.374858
2,nAH1.3,I520T,8.700000,12.306636,False,0.006834,0.402963,1272.988229,30.540397
3,nAH1.3,L184H,0.010802,0.621542,False,0.006834,0.402963,1.580623,1.542432
4,nAH1.3,P185D,10.000000,24.938619,True,0.006834,0.402963,1463.204861,61.888182
5,nAH1.3,Q450S,0.006540,0.522759,False,0.006834,0.402963,0.956929,1.297289
6,nAH1.3,W519T,0.007923,0.525908,False,0.006834,0.402963,1.159260,1.305105
7,nAH1.3,Y445M,0.006015,0.403860,False,0.006834,0.402963,0.880082,1.002226


In [92]:
fold_change_chart = (
    alt.Chart(fold_changes)
    .encode(
        x=alt.X(
            "measured_fold_change",
            title="measured fold change IC50",
            scale=alt.Scale(type="log"),
        ),
        y=alt.Y(
            "predicted_fold_change",
            title="predicted fold change IC50",
            scale=alt.Scale(type="log"),
        ),
        facet=alt.Facet("antibody", columns=4, title=None),
        color=alt.Color("lower_bound", title="lower bound"),
        tooltip=[
            alt.Tooltip(c, format=".3g") if fold_changes[c].dtype == float
            else c
            for c in fold_changes.columns.tolist()
        ],
    )
    .mark_circle(filled=True, size=60, opacity=0.6)
    .configure_axis(grid=False)
    .resolve_scale(y="independent", x="independent")
    .properties(width=150, height=150)
)

fold_change_chart

alt.Chart(...)